<a href="https://colab.research.google.com/github/RePrete/economic-macro-data-screener/blob/main/sector_performance.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [4]:
pip install yfinance

     |████████████████████████████████| 63 kB 1.4 MB/s 
     |████████████████████████████████| 6.4 MB 10.9 MB/s 
  Attempting uninstall: requests
    Found existing installation: requests 2.23.0
    Uninstalling requests-2.23.0:
      Successfully uninstalled requests-2.23.0
  Attempting uninstall: lxml
    Found existing installation: lxml 4.2.6
    Uninstalling lxml-4.2.6:
      Successfully uninstalled lxml-4.2.6
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
google-colab 1.0.0 requires requests~=2.23.0, but you have requests 2.27.1 which is incompatible.
datascience 0.10.6 requires folium==0.2.1, but you have folium 0.8.3 which is incompatible.


In [5]:
import yfinance as yf
import pandas as pd
import matplotlib.pyplot as plt

In [6]:
symbols = {
    "XLF": "Financial", 
    "XLI": "Industrial", 
    "XLE": "Energy", 
    "XLB": "Materials", 
    "XLK": "Technology", 
    "XLY": "Consumer discretionary", 
    "XLV": "Healthcare", 
    "XLP": "Consumer staples", 
    "XLU": "Utilities", 
    "XLRE": "Real estate", 
    "XLC": "Communication services", 
}
benchmark = "SPY"
timeframes = ["1d", "7d", "1mo", "3mo"]

In [7]:
symbols = {
    "IYF": "Financial", 
    "VIS": "Industrial", 
    "VDE": "Energy", 
    "VAW": "Materials", 
    "IGV": "Technology", 
    "IYC": "Consumer discretionary", 
    "VHT": "Healthcare", 
    "VDC": "Consumer staples", 
    "VPU": "Utilities", 
    "VNQ": "Real estate", 
    "XTL": "Telecom", 
}
benchmark = "SPY"
timeframes = ["1d", "7d", "1mo", "3mo"]

In [8]:
def calculate_percentage_change(open, close):
  return (close - open) / open * 100

def get_symbol_percentage_change(data, symbol, timeframe):
  ticker = data.tickers[symbol]
  history = ticker.history(period="3mo", interval="1d")
  history['Open'] = history['Close'].shift(1)
  if (timeframe == "1d"):
    open = history.tail(2).iloc[-1]['Open']
    close = history.tail(2).iloc[-1]['Close']
  elif (timeframe == "7d"):
    open = history.tail(7).iloc[0]['Open']
    close = history.iloc[-1]['Close']
  elif (timeframe == "1mo"):
    open = history.tail(30).iloc[0]['Open']
    close = history.iloc[-1]['Close']
  elif (timeframe == "3mo"):
    open = history.iloc[1]['Open']
    close = history.iloc[-1]['Close']
  
  return calculate_percentage_change(open, close)


In [9]:
data = yf.Tickers(list(symbols.keys()))

df = pd.DataFrame(columns=timeframes + ["ticker"])
for symbol in data.tickers:
  ticker_changes = {'ticker': symbols[symbol]}
  for timeframe in timeframes:
    ticker_changes[timeframe] = get_symbol_percentage_change(data, symbol, timeframe)
  df = df.append(ticker_changes, ignore_index=True)

In [10]:
def compare_with_benchmark(data, benchmark):
  data['1d'] = data['1d']/data.loc[benchmark]['1d']
  data['7d'] = data['7d']/data.loc[benchmark]['7d']
  data['1mo'] = data['1mo']/data.loc[benchmark]['1mo']
  data['3mo'] = data['3mo']/data.loc[benchmark]['3mo']

In [11]:
import plotly.express as px
from plotly.subplots import make_subplots
import plotly.graph_objects as go

In [12]:
fig = make_subplots(rows=2, cols=2, subplot_titles=['1d', '7d', '1m', '3m'])
df1d = df[['ticker', '1d']].sort_values('1d')
df7d = df[['ticker', '7d']].sort_values('7d')
df1mo = df[['ticker', '1mo']].sort_values('1mo')
df3mo = df[['ticker', '3mo']].sort_values('3mo')

fig.append_trace(
   go.Bar(name='1d', x=df1d['1d'], y=df1d['ticker'], orientation='h', marker=dict(color=df1d['1d'], colorscale=['#c62828', '#fffde7', '#558b2f'])), 
   row=1, col=1
)
fig.append_trace(
   go.Bar(name='7d', x=df7d['7d'], y=df7d['ticker'], orientation='h', marker=dict(color=df7d['7d'], colorscale=['#c62828', '#fffde7', '#558b2f'])), 
   row=1, col=2
)
fig.append_trace(
   go.Bar(name='1mo', x=df1mo['1mo'], y=df1mo['ticker'], orientation='h', marker=dict(color=df1mo['1mo'], colorscale=['#c62828', '#fffde7', '#558b2f'])), 
   row=2, col=1
)
fig.append_trace(
   go.Bar(name='3mo', x=df3mo['3mo'], y=df3mo['ticker'], orientation='h', marker=dict(color=df3mo['3mo'], colorscale=['#c62828', '#fffde7', '#558b2f'])), 
   row=2, col=2
)
# Change the bar mode
fig.update_layout(barmode='group', width=1800, height=1000)
fig.update(layout_showlegend=False)
fig.show()